<a href="https://colab.research.google.com/github/arindam15964/mlrepo1/blob/master/mutipleclassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#importing modules
import keras
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
import cv2
import numpy as np
import numpy as np
from keras.preprocessing import image
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
import os
import matplotlib.pyplot as plt
import tensorflow as tf
IMG_SIZE=256

In [0]:
# IMG_SIZE=128
# sath="/content/test/0.jpg"
# img_array=cv2.imread(sath,cv2.IMREAD_GRAYSCALE)
# new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
# plt.imshow(new_array,"gray")
# plt.show()


In [0]:
#creating train data
import random
id_array=["0","1","2","3","4"]
train_data=[]
path2="/content/train"
def create_train_data():
  for category in id_array:
    newpath=os.path.join(path2,category)
    class_num=id_array.index(category)
    for img in os.listdir(newpath):
        img_array=cv2.imread(os.path.join(newpath,img),cv2.IMREAD_COLOR )
        new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
        #plt.imshow(new_array,cmap="gray")
        #plt.show()
        #break
        train_data.append([new_array,class_num])
create_train_data()
random.shuffle(train_data)

In [10]:
len(train_data)

1659

In [11]:
X=[]
y=[]
i=0  
for features , labels in train_data:
  X.append(features)
  y.append(labels)
depth = 5
print(len(X))
one_hot_y=tf.one_hot(y, depth)
ohy=one_hot_y.numpy()
X=np.array(X).reshape(-1,IMG_SIZE,IMG_SIZE,3)
X=X/255.0

1659


In [12]:
X.shape

(1659, 256, 256, 3)

In [0]:
#creating the model
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape = (IMG_SIZE,IMG_SIZE, 3), activation = 'relu'))


model.add(Conv2D(32, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(32, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(32, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(32, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(32, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Flatten())

model.add(Dense(units = 128, activation = 'relu'))
# model.add(Dense(units = 16, activation = 'relu'))
model.add(Dense(units = 5, activation = 'softmax'))


In [0]:
# Compiling the CNN
# opt = keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X,ohy,batch_size=64,epochs=100,validation_split=0.1)

Train on 1493 samples, validate on 166 samples
Epoch 1/100
1493/1493 [==============================] - 284s 190ms/step - loss: 1.5384 - accuracy: 0.3115 - val_loss: 1.4734 - val_accuracy: 0.3675
Epoch 2/100
1493/1493 [==============================] - 286s 192ms/step - loss: 1.4483 - accuracy: 0.3677 - val_loss: 1.4507 - val_accuracy: 0.3735
Epoch 3/100
1493/1493 [==============================] - 286s 192ms/step - loss: 1.4411 - accuracy: 0.3644 - val_loss: 1.3938 - val_accuracy: 0.3494
Epoch 4/100
1493/1493 [==============================] - 283s 189ms/step - loss: 1.3541 - accuracy: 0.4300 - val_loss: 1.2926 - val_accuracy: 0.4699
Epoch 5/100
1493/1493 [==============================] - 286s 191ms/step - loss: 1.3294 - accuracy: 0.4273 - val_loss: 1.2993 - val_accuracy: 0.4458
Epoch 6/100
1493/1493 [==============================] - 283s 190ms/step - loss: 1.3118 - accuracy: 0.4481 - val_loss: 1.2866 - val_accuracy: 0.4759
Epoch 7/100
1493/1493 [==============================] - 28

In [0]:
# #creating training data generator
# train_datagen = ImageDataGenerator(
#     rescale = 1./255,
#     shear_range = 0.2,
#     zoom_range = 0.2,
#     horizontal_flip = True
# )

# train_generator = train_datagen.flow_from_directory(
#     directory=r"/content/train",
#     target_size=(IMG_SIZE, IMG_SIZE),
#     color_mode="rgb",
#     batch_size=32,
#     class_mode="categorical",
#     shuffle=True,
#     seed=42
# )

In [0]:
# #Fitting the traing data
# model.fit_generator(
#     train_generator,
#     epochs=5,
#     steps_per_epoch =  1659,
# )

In [0]:
# Predicting USing the model
sdf=pd.read_csv("/content/sample.csv")
img_array=sdf['image']
pathf="/content/test/"
y_f=[]
for image1 in img_array:
  newpath=pathf+image1
  img=image.load_img(newpath,target_size=(IMG_SIZE,IMG_SIZE))
  img=image.img_to_array(img)
  img=np.expand_dims(img,axis=0)
  img=img/255
  value=model.predict(img)
  y_f.append(value)
  z_f=[]
for a in y_f:
  for data in a:
    z=np.argmax(data, axis=None)
    z_f.append(z)
    
  
dict={
    'image': img_array,
    'label': z_f
}
dff=pd.DataFrame.from_dict(dict)

In [0]:
dff.to_csv("ans.csv",index=False)
print(dff)